# are discriminated unions like Typescript literal types?

Let us see whether there is an F# equivalent to the following:

```typescript
/**
 * styling variables for this App
 */
export type YouTubeCssVariables =

    /* html anchor */
    '--thumbs-header-link-color' |
    '--thumbs-header-link-text-decoration' |

    /* thumbs set */
    '--thumbs-set-header-color' |
    '--thumbs-set-background-color' |
    '--thumbs-set-header-position' |
    '--thumbs-set-padding-top'
    ;

/**
 * defines a name-value pair for a CSS variable
 */
export interface YouTubeCssOption {
    /**
     * CSS variable name
     */
    variableName: YouTubeCssVariables;

    /**
     * CSS variable value
     */
    variableValue: string;
}
```

With my current level of ignorance, I assume that the F# equivalent is the following type, a discriminated with three members:

In [ ]:
#!fsharp

type YouTubeCssVariable =
    | ThumbsHeaderLinkColor of string
    | ThumbsHeaderLinkTextDecoration of string
    | ThumbsSetHeaderColor of string
    | ThumbsSetBackgroundColor of string
    | ThumbsSetHeaderPosition of string
    | ThumbsSetPaddingTop of string

    member this.Name =
        match this with
        | ThumbsHeaderLinkColor _ -> "--thumbs-header-link-color"
        | ThumbsHeaderLinkTextDecoration _ -> "--thumbs-header-link-text-decoration"
        | ThumbsSetHeaderColor _ -> "--thumbs-set-header-color"
        | ThumbsSetBackgroundColor _ -> "--thumbs-set-background-color"
        | ThumbsSetHeaderPosition _ -> "--thumbs-set-header-position"
        | ThumbsSetPaddingTop _ -> "--thumbs-set-padding-top"

    member this.Value =
        match this with
        | ThumbsHeaderLinkColor v -> v
        | ThumbsHeaderLinkTextDecoration v -> v
        | ThumbsSetHeaderColor v -> v
        | ThumbsSetBackgroundColor v -> v
        | ThumbsSetHeaderPosition v -> v
        | ThumbsSetPaddingTop v -> v

    member this.Pair =
        let n = this.Name
        let v = this.Value
        n, v


This single, `YouTubeCssVariable` type is the equivalent of the [literal type](https://www.typescriptlang.org/docs/handbook/2/everyday-types.html#literal-types), `YouTubeCssVariables` _and_ the interface, `YouTubeCssOption`, we have in Typescript. This means that the equivalent assignment in Typescript:

```typescript
{
    variableName: '--thumbs-set-background-color',
    variableValue: 'inherit'
}
```

…looks like this in F#:

In [ ]:
#!fsharp

let thumbsSetHeaderColor = ThumbsSetBackgroundColor "inherit"

thumbsSetHeaderColor.Pair |> printf "%A"

("--thumbs-set-background-color", "inherit")

## the Tomas Petricek approach

Tomas Petricek was kind enough to take my question seriously and [answer it](https://stackoverflow.com/a/71528232/22944). His approach is to resort to .NET Reflection (for the sake of maintainable code). So, with his guidance, I would start with his general-purpose format function and another rendition of `YouTubeCssVariable`:

In [ ]:
#!fsharp

open System
open Microsoft.FSharp.Reflection

let formatCase<'T> () =
  let formatName (s:string) =
    [ yield "-"
      for c in s do
        if Char.IsUpper(c) then yield "-"
        yield c.ToString().ToLower() ]
    |> String.concat ""
  let lookup = 
    FSharpType.GetUnionCases(typeof<'T>)
    |> Seq.map (fun c -> FSharpValue.MakeUnion(c, [||]), formatName c.Name)
    |> dict
  fun (c:'T) -> lookup.[c]

type YouTubeCssVariable =
    | ThumbsHeaderLinkColor
    | ThumbsHeaderLinkTextDecoration
    | ThumbsSetHeaderColor
    | ThumbsSetBackgroundColor
    | ThumbsSetHeaderPosition
    | ThumbsSetPaddingTop

## my post-Tomas approach

Tomas is basically not telling me directly that there is no direct translation from Typescript to F# as he concentrates solely on using Reflection to convert a Pascal-case string to a snake-case string:

In [ ]:
#!fsharp

let f = formatCase<YouTubeCssVariable>()

f ThumbsSetBackgroundColor

--thumbs-set-background-color

It seems clear to me (today) that I could use `nameof` to accomplish the same thing:

In [ ]:
#!fsharp

nameof ThumbsSetBackgroundColor

ThumbsSetBackgroundColor

Now that we have a Pascal-case string we can use a utility like my `toKebobCase` method [[GitHub](https://github.com/BryanWilhite/Songhay.Modules/blob/main/Songhay.Modules/StringUtility.fs#L109)] (in C#). In fact, the `formatName` function from Tomas is very much like my `toKebobCase` method—so I will appropriate it with a more general-purpose name:

In [ ]:
#!fsharp

let toKebobCase (s: string) =
    let processChar i c =
        if (i > 0) && Char.IsUpper(c) then $"-{Char.ToLower(c)}"
        else $"{Char.ToLower(c)}"
    let stringArray = s.ToCharArray() |> Array.mapi processChar
    String.Join(String.Empty, stringArray)

"ThumbsSetBackgroundColor" |> toKebobCase

thumbs-set-background-color

Now I can use my `toKebobCase` function to eliminate the magic-string smell:

In [ ]:
#!fsharp

type YouTubeCssVariable =
    | ThumbsHeaderLinkColor of string
    | ThumbsHeaderLinkTextDecoration of string
    | ThumbsSetHeaderColor of string
    | ThumbsSetBackgroundColor of string
    | ThumbsSetHeaderPosition of string
    | ThumbsSetPaddingTop of string

    member this.Name =
        let name =
            match this with
            | ThumbsHeaderLinkColor _ -> nameof ThumbsHeaderLinkColor |> toKebobCase
            | ThumbsHeaderLinkTextDecoration _ -> nameof ThumbsHeaderLinkTextDecoration |> toKebobCase
            | ThumbsSetHeaderColor _ -> nameof ThumbsSetHeaderColor |> toKebobCase
            | ThumbsSetBackgroundColor _ -> nameof ThumbsSetBackgroundColor |> toKebobCase
            | ThumbsSetHeaderPosition _ -> nameof ThumbsSetHeaderPosition |> toKebobCase
            | ThumbsSetPaddingTop _ -> nameof ThumbsSetPaddingTop |> toKebobCase
        $"--{name}"

    member this.Value =
        match this with
        | ThumbsHeaderLinkColor v -> v
        | ThumbsHeaderLinkTextDecoration v -> v
        | ThumbsSetHeaderColor v -> v
        | ThumbsSetBackgroundColor v -> v
        | ThumbsSetHeaderPosition v -> v
        | ThumbsSetPaddingTop v -> v

    member this.Pair =
        let n = this.Name
        let v = this.Value
        n, v


`YouTubeCssVariable` is now slightly better but still super verbose (but also very clear and open to change).

In my ignorance, I would like to see:

- F# `match` cases support generics
- treat discriminated union members like data

In the mean time, I can return to the `printf` expression from above with a slightly better `YouTubeCssVariable` type:

In [ ]:
#!fsharp

let thumbsSetHeaderColor = ThumbsSetBackgroundColor "inherit"

thumbsSetHeaderColor.Pair |> printf "%A"

("--thumbs-set-background-color", "inherit")

## wait a minute: can we use `.ToString` instead of `nameof`?

Let me step back a minute and look at a simple discriminated union:

In [ ]:
#!fsharp

type SimpleThree =
    | One
    | Two
    | Three

let three = Three

three.ToString()

Three

Ouch! All of that writing of mine above might be seriously moot. This could be my next clue:

In [ ]:
#!fsharp

type SimpleFour =
    | One
    | Two
    | Three
    | Four

    member this.Value = this.ToString().ToLowerInvariant()

let four = Four

four.Value

four

I now see that `SimpleFour` is very, very like those TypeScript literal types! I can take this information and go back to `YouTubeCssVariable`:

In [ ]:
#!fsharp

open System

type YouTubeCssVariable =
    | ThumbsHeaderLinkColor of string
    | ThumbsHeaderLinkTextDecoration of string
    | ThumbsSetHeaderColor of string
    | ThumbsSetBackgroundColor of string
    | ThumbsSetHeaderPosition of string
    | ThumbsSetPaddingTop of string

    member this.Name =
        let name = this.ToString().Split(' ')[0] |> toKebobCase
        $"--{name}"

    member this.Value =
        (this.ToString().Split(' ')[1]).Replace("\"", String.Empty)

    member this.Pair =
        let n = this.Name
        let v = this.Value
        n, v

let thumbsSetHeaderColor = ThumbsSetBackgroundColor "inherit"

thumbsSetHeaderColor.Pair |> printf "%A"

("--thumbs-set-background-color", "inherit")

This is finally down to a size that is most comfortable to me! My only concern is having to resort to `.Replace("\"", String.Empty)` which is putting my full trust in how F# returns the string representation of one case of a <acronym title="Discriminated Union">DU</acronym>. Will it change in the next six months? Probably not.

[Bryan Wilhite is on LinkedIn](https://www.linkedin.com/in/wilhite)🇺🇸💼